# Proyek Analisis Data: Penggunaan Sepeda Berdasarkan Kelembapan dan RFM

- **Nama:** Abi Eka Putra Wulyono
- **Email:** abiekapw@gmail.com
- **ID Dicoding:** abi_eka_putra_we6p

## 1. Menentukan Pertanyaan Bisnis

### Pertanyaan 1: Bagaimana pengaruh kelembapan terhadap jumlah penyewaan sepeda?

### Pertanyaan 2: Bagaimana perilaku penyewaan sepeda dapat diukur menggunakan analisis RFM (Recency, Frequency, Monetary)?

## 2. Import Semua Packages/Library yang Digunakan

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

## 3. Data Wrangling

### Gathering Data
Dataset penggunaan sepeda diambil dari sumber terbuka yang menyediakan data harian (`day.csv`) dan data jam (`hour.csv`).

In [ ]:
# Memuat dataset
day_data = pd.read_csv('data/day.csv')
hour_data = pd.read_csv('data/hour.csv')

# Menampilkan beberapa data teratas
day_data.head(), hour_data.head()

### Insight:


- Dataset berisi informasi harian dan per jam terkait faktor cuaca, kelembapan, serta jumlah pengguna terdaftar dan kasual.
- Terdapat sekitar 730 baris data harian dan 17379 baris data jam yang mencakup informasi selama 2 tahun.
    

## 4. Assessing Data

In [ ]:
# Memeriksa nilai yang hilang dan tipe data
day_data.info()
hour_data.info()  

### Insight:


- Tidak ada nilai yang hilang dalam dataset.
- Tipe data untuk masing-masing kolom sudah sesuai.
    

## 5. Cleaning Data

In [ ]:
# Menghapus kolom yang tidak relevan dan mengecek duplikasi
day_data_cleaned = day_data.drop(columns=['instant'])
hour_data_cleaned = hour_data.drop(columns=['instant'])

# Menghapus duplikasi
day_data_cleaned = day_data_cleaned.drop_duplicates()
hour_data_cleaned = hour_data_cleaned.drop_duplicates()

day_data_cleaned.head()

### Insight:


- Kolom 'instant' telah dihapus, dan tidak ditemukan data duplikat.
- Data siap digunakan untuk analisis lebih lanjut.
    

## 6. Exploratory Data Analysis (EDA)

### Explore: Pengaruh Kelembapan Terhadap Penyewaan Sepeda

In [ ]:
# Menambahkan kategori kelembapan berdasarkan nilai 'hum'
bins = [0, 0.3, 0.6, 1.0]
labels = ['Kering', 'Normal', 'Lembap']
hour_data_cleaned['humidity_category'] = pd.cut(hour_data_cleaned['hum'], bins=bins, labels=labels)

# Mengelompokkan data berdasarkan kelembapan dan menghitung total penyewaan
humidity_group = hour_data_cleaned.groupby('humidity_category', observed=True).agg(total_usage=('cnt', 'sum')).reset_index()

# Visualisasi kategori kelembapan
sns.set_style('whitegrid')
plt.figure(figsize=(10, 6))
sns.barplot(data=humidity_group, x='humidity_category', y='total_usage', hue='humidity_category', palette='coolwarm', legend=False)
plt.title('Total Penggunaan Sepeda Berdasarkan Kelembapan', fontsize=14)
plt.xlabel('Kategori Kelembapan', fontsize=12)
plt.ylabel('Total Penggunaan Sepeda', fontsize=12)
plt.show()

### Insight:


- Kelembapan normal (30-60%) memiliki tingkat penyewaan sepeda tertinggi.
- Kelembapan yang terlalu rendah atau terlalu tinggi tidak ideal untuk penyewaan sepeda.
    

## 7. Visualization & Explanatory Analysis

### Pertanyaan 1: Bagaimana pengaruh kelembapan terhadap jumlah penyewaan sepeda?

In [ ]:
# Visualisasi pengaruh kelembapan
humidity_group.plot(kind='bar', x='humidity_category', y='total_usage')

### Pertanyaan 2: Bagaimana perilaku penyewaan sepeda dapat diukur menggunakan analisis RFM?

In [ ]:
# Analisis RFM
current_date = pd.to_datetime(max(day_data_cleaned['dteday']))

# Menghitung Recency, Frequency, Monetary
rfm_data = day_data_cleaned.groupby('registered').agg({
    'dteday': lambda x: (pd.to_datetime(current_date) - pd.to_datetime(x.max())).days,  # Recency
    'cnt': 'sum',  # Monetary
    'instant': 'count'  # Frequency
}).reset_index()

# Menampilkan data RFM
rfm_data.columns = ['registered', 'Recency', 'Monetary', 'Frequency']
rfm_data.head()

### Insight:


- Pengguna terdaftar menyumbang lebih banyak penyewaan dan memiliki frekuensi yang lebih tinggi dibandingkan pengguna kasual.
    

## 8. Analisis Lanjutan

In [ ]:
# Contoh model prediksi dengan Linear Regression
X = day_data_cleaned[['hum', 'temp']]
y = day_data_cleaned['cnt']

# Memisahkan data untuk training dan testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Membuat dan melatih model
model = LinearRegression()
model.fit(X_train, y_train)

# Prediksi dan evaluasi
predictions = model.predict(X_test)
print(predictions)

## 9. Conclusion


- **Pertanyaan 1:** Kelembapan yang lebih normal memberikan kondisi terbaik untuk penyewaan sepeda.
- **Pertanyaan 2:** Analisis RFM menunjukkan bahwa pengguna terdaftar lebih aktif dalam hal frekuensi penyewaan dan memberikan kontribusi yang lebih besar terhadap total penyewaan.
    